# Lab 3: Modelling radiation imbalance

The goal of this lab is to use simple assumptions and a few equations, with help from Python, to reason about the Earth system.  We will be building our first **model**.  

Our science questions are:
- How does the Earth's global mean temperature adjust to a radiation imbalance?
- By how much would the temperature change for a given change in the Earth system?  For example
    - changes in greenhouse gas concentration
    - changes in cloudiness
- What is the role of ocean heat capacity in climate change?

We will make use of the Trenberth and Fasullo (2012) summary plot of Earth's radiation budget.  Double-click on the image to see how it was added to the Jupyter notebook--this will help you on Assignment 2.

![Observed global energy flows from Trenberth and Fasullo (2012)](./figs/radbudget-trenberth_fasullo.jpeg)

What are the components we see here?

The incoming solar radiation or **insolation**, $Q$, is what we receive from the Sun.

The **Absorbed Shortwave Radiation** or ASR is the part of the incoming sunlight that is *not* reflected back to space, i.e. that part that is absorbed somewhere within the Earth system.

Mathematically we write

$$ \text{ASR} = Q - F_{reflected} = (1-\alpha) Q $$

where $F_{\text{reflected}}$ is the quantity of radiation reflected, and $\alpha$ is the planetary **albedo**.

The **Outgoing Longwave Radiation** or OLR depends on Earth's surface temperature, according to the Stefan-Boltzmann law.

In class we found that blackbody radiation was not a very good fit to the *observed* temperature due to the influence of our greenhouse atmosphere.  To account for the atmosphere, we introduced the  **transmissivity**  of the atmosphere, $\tau$, a number less than 1 that represents the greenhouse effect of Earth's atmosphere.  

With the transmissivity included, the OLR is:

$$ \text{OLR} = \tau \sigma T_s^4 $$

where $\sigma$ is the Stefan-Boltzmann constant and $T_s$ is the surface temperature. 

The Earth system is in **energy balance** when energy in = energy out, i.e. when

$$ \text{ASR} = \text{OLR} $$

When this is not the case, there must be some net energy gained or lost by the system over time.

$$
\begin{align} 
\text{ASR} - \text{OLR} &= \text{flux in – flux out} \\
    &= \text{net energy flux in to system} \\
    &= \frac{dE}{dt} \\
\end{align}
$$

where $E$ is the **total heat content** ("enthalpy") of the Earth system.  

This is a general statement of the energy budget.  To make a *model*, we want to simulate changes in some measurable variable over time.  Here we will make a model for temperature change.

### An energy balance **model**

If we now suppose that 

$$ E = C T_s $$

where $E$ is the total heat content as above, $T_s$ is the global mean surface temperature, and $C$ is a constant – the **effective heat capacity** of the atmosphere- ocean column, in units of $\text{J m}^{-2} \text{K}^{-1}$.

then we can rewrite our energy budget equation as

$$ C \frac{dT}{dt} = \text{ASR} - \text{OLR} $$

We need to make a decent assumption about the effective heat capacity of the Earth system to use this equation.  For now, because the oceans are a dominant control on global heat capacity, we are going to use a value of C equivalent to heating 100 meters of water:

$$C = c_w \rho_w H$$

where 

$c_w = 4 \times 10^3$ J kg$^{-1}$ $^\circ$C$^{-1}$ is the specific heat of water,

$\rho_w = 10^3$ kg m$^{-3}$ is the density of water, and

$H$ is an effective depth of water that is heated or cooled.

In [ ]:
c_w = 4E3  #  Specific heat of water in J/kg/K
rho_w = 1E3  #  Density of water in kg/m3
H = 100.   #  Depth of water in m
C = c_w * rho_w * H   #  Heat capacity of the model 

***
## Part 1: Demonstration and skills practice

In [ ]:
import numpy as np
## a special instruction for the Jupyter notebook
##   Display all plots inline in the notebook
%matplotlib inline  
##  import the plotting package
import matplotlib.pyplot as plt

### 1. Fitting parameters to observations

We introduced transmissivity, $\tau$, into an equation based on the Stefan-Boltzmann law to account for the atmospheric greenhouse trapping some outgoing longwave radiation.

$$ \text{OLR} = \tau \sigma T_s^4 $$

We asserted $\tau$ was small and positive, between 0 and 1.  What is a realistic value of $\tau$?  We can deduce one from observations.

**Demonstration: Finding a value for $\tau$ that fits the observations of $OLR$ and $T_s$.**

Let's look at Trenberth and Fasullo for the observed outgoing longwave.  We know the global average surface temperature is about 15 Celsius, or 288 K.

In [ ]:
## Lizz will enter live python code here

We can deduce the albedo in a similar way.
From the Trenberth and Fasullo plot above, the area-averaged incoming solar radiation or **insolation** is 341.3 W m$^{-2}$.

Let's denote this quantity by $Q$.

In [ ]:
Q = 341.3  ## the insolation, in W/m2

#### *Exercise*:
Based on the Trenberth and Fasullo plot, compute the global mean albedo.  Assign it to a variable named `alpha` when you compute it.

*Hint:* refer back to where we found the albedo in the equation for absorbed shortwave radiation:
$$ \text{ASR} = Q - F_{reflected} = (1-\alpha) Q $$

In [ ]:
## your code here

***
### 2. Reusable functions for fun and profit

Let's look again at the energy balance equation:
$$ \text{ASR} = \text{OLR} $$

Because OLR depends on temperature, we can identify what surface temperature is required for energy balance (the **equilibrium temperature**).  Let's use the expressions for ASR and OLR we have already identified, replacing ASR on the left hand side and OLR on the right hand side:

$$
\begin{align} 
\text{ASR} &= \text{OLR} \\
(1-\alpha) Q &= \tau \sigma T_s^4 \\
\frac{(1-\alpha) Q}{\tau \sigma} &= T_s^4 \\
\end{align}
$$

And finally
$$  T   = \left(\frac{(1-\alpha) Q}{\tau \sigma} \right)^{1/4} $$

#### *Exercise*: 
Write a reusable function called `equilibrium_temperature` that computes the equilibrium temperature, given albedo and transmissivity as inputs.

In [ ]:
## your code here

#### Default values
We can make our function even easier to use by adding *default values* for the parameters.  These values give a baseline assumption that we can use in general, or choose to modify it when we want to change the assumption.

In [ ]:
def StefanBoltzmann(T, sigma):
    P = sigma * (T**4) ## Power per unit area, in W/m2
    return P

## Lizz demonstrate how to add default values here

#### *Exercise*: 
Revise your reusable function for equilibrium temperature to include default values for $\sigma$ and for Earth's surface temperature.

In [ ]:
## your code here

***
### 3. Timestepping

The derivative is the *instantaneous rate of change*. It is defined as 

$$ \frac{dT}{dt} = \lim_{\Delta t\rightarrow 0}⁡ \frac{\Delta T}{\Delta t}$$

- On the computer there is no such thing as an instantaneous change. 
- We are always dealing with *discrete quantities*.
- We approximate the derivative with $\Delta T/ \Delta t$. 
- The time interval $\Delta t$ is called the **timestep**, the smallest unit of time captured in our model.  So long as we keep $\Delta t$ "small enough", the approximation is valid and useful.

So, we can re-write the model from the intro as

$$ C  \frac{\Delta T}{\Delta t} \approx \text{ASR} - \text{OLR}$$

where $\Delta T$ is the **change in temperature predicted by our model** over a short time interval $\Delta t$.

We can immediately answer a question based on this equation!  Given a current temperature $T_1$ at time $t_1$, and today's observed energy imbalance, what is the temperature $T_2$ at a future time $t_2$?

We can write

$$ \Delta T = T_2-T_1 $$
$$ \Delta t = t_2-t_1 $$

and so our model says

$$ C  \frac{T_2-T_1}{\Delta t} = \text{ASR} - \text{OLR} $$

Which we can rearrange to **solve for the future temperature**:

$$ T_2 = T_1 + \frac{\Delta t}{C} \left( \text{ASR} - \text{OLR}(T_1) \right)  $$

**Demonstration: Computing temperature one year in the future.**

In [ ]:
## Lizz to write live code here
##REMOVE BELOW
dt = 60. * 60. * 24. * 365.   # one year expressed in seconds

T1 = 288
energy_imbalance = 0.9 ## W/m2, the net energy imbalance we have computed

T2 = T1 + ((dt/C)*(energy_imbalance))

In [ ]:
T2

In [ ]:
def step_forward(T, alpha=0.3, tau=0.6):
    T_next = T + dt / C * (energy_imbalance)
    return T_next

#### *Exercise*:
Assume that the concentration of CO$_2$ in the atmosphere is increasing linearly in time at rate W:
$$ \frac{dCO_2}{dt} = W $$

$$ \frac{\Delta CO_2}{\Delta t} \approx W $$ 

$$ (CO_2(t_1) - CO_2(t_0)) \approx W (t_1-t_0) $$

Write a function to compute the CO$_2$ concentration at time t$_1$, using a timestep $dt$=1 year, and an initial CO2 concentration input to the function.

In [ ]:
## your code here

***
### 4. For-loops

Often we use a `for` loop to go through a list of items and perform some action.

These loops are generally constructed by:

- Initializing one or more variables before the loop starts
- Performing some computation on each item in the loop body, possibly changing the variables in the body of the loop
- Looking at the resulting variables when the loop completes

We will use a list of numbers to demonstrate the concepts and construction of these loop patterns.

In [ ]:
my_values = [5, 10, 15, 20]

for m in my_values:
    print(m)

We can run the loop over a list of items, or we can set up a *counter* within a specified range.  This is very useful if we need to perform an action a set number of times, such as sampling an experiment 10 or 100 times.

Here we will use `range(end_value)` to define the counting range.

In [ ]:
for i in range(10):
    print(i)

When we are performing some action in the loop, we often want to store the result.  We can set up an empty list or array, then update it within the loop.


In [ ]:
my_output = np.zeros(5)

for i in range(5):
    value = i * 3
    my_output[i] = value

print(my_output)

#### *Exercise*:
Use `np.zeros` to define an empty list 10 entries long.  Write a for-loop that goes through the list and stores values from 1 to 10 in order.  Print to show the result.

In [ ]:
## your code here

#### *Exercise*:
Complete the following code to compute 10 future values of CO$_2$ concentration, assuming the same linear increase we used above.

In [ ]:
numsteps = 10

CO2_steps = np.zeros(numsteps+1)
Years = np.zeros(numsteps+1)

CO2_steps[0] = 400. 
for n in range(numsteps):
    Years[n+1] = n+1
#    CO2_steps[n+1] = ...
## uncomment and complete the line above
print(CO2_steps)

***
### 5. A basic plot

Last week we plotted `xarray` objects using a built-in `.plot()` command.  That will not work for the `numpy` objects we are working with today.  See what happens:

In [ ]:
CO2_steps.plot()

Instead, we will use the more general plotting library [matplotlib](https://matplotlib.org/index.html).  Refer back to Lab 1 for some basic discussion of plotting with matplotlib.  The general steps are:
- remember to import matplotlib
- set up the axes
- use `plt.plot(xvalues, yvalues)` to plot your data on the axes
- label the axes using commands such as `plt.xlabel()`

In [ ]:
import matplotlib.pyplot as plt

x = np.linspace(0, 2)

plt.plot(x, x, label='f(x) = x')
plt.plot(x, x**2, label='f(x) = x$^{2}$')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.legend(loc='best');

Let's modify that code to plot our `CO2_steps` array instead.

In [ ]:
## Lizz to enter live code here

***
## Part 2: Computing temperature change in response to energy imbalance

Suppose that, due to changes in atmospheric composition and subsequent changes in cloudiness:

- The longwave transmissitivity decreases to $\tau = 0.57$

- The planetary albedo increases to $\alpha = 0.32$ 

This is our first climate-change scenario!  We can use the tools we have developed here to analyse resulting changes in temperature.

### Lab Procedure
1. Use the `equilibrium_temperature` function you defined above to compute the equilibrium temperature for this new climate scenario.
2. Define a reusable function called `ASR` that computes the absorbed shortwave radiation, and a function called `OLR` that computes the outgoing longwave radiation, for any values of albedo, transmissivity, solar flux, and temperature.  Define and use appropriate default values for the arguments.
    - Compute the instantaneous ASR and OLR with these new values of albedo and transmissivity, and today's observed temperature.
3. Modify the `step_forward` function we defined above to evolve temperature in time based on ASR and OLR imbalance (rather than a hard-coded `energy_imbalance`).
4. Use a for-loop to step the temperature 50 years into the future.
5. Address the science question by making a plot of temperature over the 50-year simulation.

In [ ]:
## Add cells of code and markdown as needed here to complete the lab

- *Optional extension: Examine the role of ocean heat capacity in the climate change model.*
    - Modify the model (likely in your time stepping function) to double the heat capacity $C$.
    - Run this new model to equilibrium and store the results in a new array
    - Compare the model with your original result
    - Interpret how the heat capacity affects climate change.

***
### Endnotes
- [Here is a tutorial on default values.](https://www.pythontutorial.net/python-basics/python-default-parameters/)
- Development: The content of this lab is based on Brian Rose's Climate Laboratory book ([landing page](https://brian-rose.github.io/ClimateLaboratoryBook/home.html); [original notebook](https://brian-rose.github.io/ClimateLaboratoryBook/courseware/zero-dim-ebm.html)).
- This lab was last updated by Lizz Ultee, 23 Feb 2024.